<h1>Gradient Boosting</h1>

In [1]:
library(mlr, warn.conflicts = F)
source('../utils.r')

set.seed(42)

folder_name = '../../raw_data' 
file_name   = 'german.csv'

Loading required package: ParamHelpers
Warning message:
“replacing previous import ‘BBmisc::isFALSE’ by ‘backports::isFALSE’ when loading ‘mlr’”

<h2>1. Dataprep</h2>

In [2]:
data = read.csv(file=sprintf('%s/%s',folder_name,file_name))

In [3]:
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))
head(data)

NRow: 1000
NCol: 22

X,V1,V2,V3,V4,V5,V6,V7,V8,V9,⋯,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21
1,A11,6,A34,A43,1169,A65,A75,4,A93,⋯,A121,67,A143,A152,2,A173,good,A192,A201,1
2,A12,48,A32,A43,5951,A61,A73,2,A92,⋯,A121,22,A143,A152,1,A173,good,A191,A201,2
3,A14,12,A34,A46,2096,A61,A74,2,A93,⋯,A121,49,A143,A152,1,A172,bad,A191,A201,1
4,A11,42,A32,A42,7882,A61,A74,2,A93,⋯,A122,45,A143,A153,1,A173,bad,A191,A201,1
5,A11,24,A33,A40,4870,A61,A73,3,A93,⋯,A124,53,A143,A153,2,A173,bad,A191,A201,2
6,A14,36,A32,A46,9055,A65,A73,2,A93,⋯,A124,35,A143,A153,1,A172,bad,A192,A201,1


<p style='color: red'> ATENTION: </p>Specifically in R, in classification problem, target must be set as Factor.

In [4]:
data$V21 = as.factor(data$V21)

MLR works only with features and target, this means that others columns must be dorped.

In [5]:
drops = c('X')
data  = data[ , !(names(data) %in% drops)]
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 21

Function
```R
createDummyFeatures(obj, target = character(0L), method = "1-of-n",  cols = NULL)
```

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/createDummyFeatures

In [24]:
data = createDummyFeatures(obj = data, target = 'V21')
cat(sprintf('NRow: %d\nNCol: %d',nrow(data), ncol(data)))

NRow: 1000
NCol: 63

<h2>2. Modeling</h2>

Function
```R
makeLearner(cl, id = cl, predict.type = "response", predict.threshold = NULL, 
            fix.factors.prediction = FALSE, ..., par.vals = list(), config = list())
```
Param.:

* cl: [character(1)] Class of learner. By convention, all classification learners start with “classif.”. A list of all integrated learners is available on the learners help page < https://mlr-org.github.io/mlr-tutorial/release/html/integrated_learners/ >.
* predict: [character(1)] “response” (= labels) or “prob” (= probabilities and labels by selecting the ones with maximal probability). Default is “response”.
* par.vals: [list] Optional list of named (hyper)parameters. The arguments in ... take precedence over values in this list. We strongly encourage you to use one or the other to pass (hyper)parameters to the learner but not both.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [7]:
cl = "classif.bst"

List all parameters that can be used in this classifier.
The value must be set in par.vals parameter.

In [8]:
getParamSet(cl)

                   Type len    Def         Constr Req Tunable Trafo
cost            numeric   -    0.5         0 to 1   -    TRUE     -
family         discrete   -      - gaussian,hinge   -    TRUE     -
mstop           integer   -     50       1 to Inf   -    TRUE     -
nu              numeric   -    0.1    -Inf to Inf   -    TRUE     -
twinboost       logical   -  FALSE              -   -    TRUE     -
f.init          untyped   - <NULL>              -   -    TRUE     -
xselect.init    untyped   - <NULL>              -   -    TRUE     -
center          logical   -  FALSE              -   -    TRUE     -
trace           logical   -  FALSE              -   -   FALSE     -
numsample       integer   -     50       1 to Inf   -    TRUE     -
df              integer   -      4       1 to Inf   -    TRUE     -
minsplit        integer   -     20       1 to Inf   -    TRUE     -
minbucket       integer   -      -       1 to Inf   -    TRUE     -
cp              numeric   -   0.01         0 to 

In [9]:
learner = makeLearner(cl = cl
                     , predict.type = "response"
                     , par.vals = list()
                     )

Function
```R
makeClassifTask(id = deparse(substitute(data)), data, target, weights = NULL, blocking = NULL, 
                positive = NA_character_, fixup.data = "warn", check.data = TRUE)
```
Param.:

* data: [data.frame] A data frame containing the features and target variable(s).
* target: [character(1)] Name of the target variable.
* positive: [character(1)] Positive class for binary classification (otherwise ignored and set to NA). Default is the first factor level of the target attribute.
* fixup.data: [character(1)] Should some basic cleaning up of data be performed? Currently this means removing empty factor levels for the columns. Possible coices are: “no” = Don't do it. “warn” = Do it but warn about it. “quiet” = Do it but keep silent. Default is “warn”.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeLearner

In [10]:
task = makeClassifTask( data = data
                      , target = 'V21'
                      , positive = '2'
                      , fixup.data = 'no'
                      )

Function:
```R
makeResampleDesc(method, predict = "test", ..., stratify = FALSE, stratify.cols = NULL)
```
Param.:

* method: [character(1)] “CV” for cross-validation, “LOO” for leave-one-out, “RepCV” for repeated cross-validation, “Bootstrap” for out-of-bag bootstrap, “Subsample” for subsampling, “Holdout” for holdout.
* predict: What to predict during resampling: “train”, “test” or “both” sets. Default is “test”.
* ... : [any] Further parameters for strategies.
    * iters [integer(1)] Number of iterations, for “CV”, “Subsample” and “Boostrap”.
    * split [numeric(1)] Proportion of training cases for “Holdout” and “Subsample” between 0 and 1. Default is 2/3.
    * reps [integer(1)] Repeats for “RepCV”. Here iters = folds * reps. Default is 10.
    * folds [integer(1)] Folds in the repeated CV for RepCV. Here iters = folds * reps. Default is 10.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/makeResampleDesc

In [11]:
resample = makeResampleDesc( method = "CV"
                           , iters = 10
                           , predict = 'both'
                           , stratify = FALSE
                           )

List of performance measures:

Doc.: http://mlr-org.github.io/mlr-tutorial/release/html/measures/

In [12]:
measures = list(mmce #MMCE 
               ,acc  #acuracia
               #,f1   #f1
               #,ppv  #precision
               #,tpr  #recall
               #,auc  #AUC
               #,gini #Gini
               ,timetrain #tempo execucao
               )

Function:
```R
resample(learner, task, resampling, measures, weights = NULL, models = FALSE, extract, 
         keep.pred = TRUE, ..., show.info = getMlrOption("show.info"))
```
Param.:

* learner: [Learner] The learner.
* task: [Task] The task.
* resampling: [ResampleInstance] Resampling strategy.
* measures: [Measure | list of Measure] Performance measure(s) to evaluate. Default is mean misclassification error (mmce)
* weights: [numeric] Optional, non-negative case weight vector to be used during fitting. If given, must be of same length as observations in task and in corresponding order. Overwrites weights specified in the task. By default NULL which means no weights are used unless specified in the task.
* models: [logical(1)] Should all fitted models be returned? Default is FALSE.
* keep.pred: [logical(1)] Keep the prediction data in the pred slot of the result object. If you do many experiments (on larger data sets) these objects might unnecessarily increase object size / mem usage, if you do not really need them. In this case you can set this argument to FALSE. Default is TRUE.
* show.info: [logical(1)] Print verbose output on console? Default is set via configureMlr.

Doc.: https://www.rdocumentation.org/packages/mlr/versions/2.10/topics/resample

In [13]:
r = resample(learner = learner
            ,task = task 
            ,resampling = resample 
            ,measures = measures
            #---------------------#
            ,models = TRUE
            ,keep.pred = FALSE
            ,show.info = TRUE
            )

[Resample] cross-validation iter 1: mmce.test.mean=0.26,acc.test.mean=0.74,timetrain.test.mean=0.092
[Resample] cross-validation iter 2: mmce.test.mean=0.32,acc.test.mean=0.68,timetrain.test.mean=0.357
[Resample] cross-validation iter 3: mmce.test.mean=0.33,acc.test.mean=0.67,timetrain.test.mean=0.198
[Resample] cross-validation iter 4: mmce.test.mean= 0.3,acc.test.mean= 0.7,timetrain.test.mean=0.086
[Resample] cross-validation iter 5: mmce.test.mean=0.29,acc.test.mean=0.71,timetrain.test.mean=0.086
[Resample] cross-validation iter 6: mmce.test.mean= 0.3,acc.test.mean= 0.7,timetrain.test.mean=0.198
[Resample] cross-validation iter 7: mmce.test.mean=0.33,acc.test.mean=0.67,timetrain.test.mean=0.088
[Resample] cross-validation iter 8: mmce.test.mean=0.26,acc.test.mean=0.74,timetrain.test.mean=0.194
[Resample] cross-validation iter 9: mmce.test.mean=0.25,acc.test.mean=0.75,timetrain.test.mean=0.085
[Resample] cross-validation iter 10: mmce.test.mean=0.28,acc.test.mean=0.72,timetrain.test.

<h2>3. Result Analysis</h2>

Train Measures

In [14]:
r$measures.train

iter,mmce,acc,timetrain
1,0.2777778,0.7222222,0.092
2,0.2877778,0.7122222,0.357
3,0.2855556,0.7144444,0.198
4,0.2877778,0.7122222,0.086
5,0.2777778,0.7222222,0.086
6,0.2855556,0.7144444,0.198
7,0.2888889,0.7111111,0.088
8,0.2855556,0.7144444,0.194
9,0.2833333,0.7166667,0.085
10,0.2855556,0.7144444,0.115


Test Measures

In [15]:
r$measures.test

iter,mmce,acc,timetrain
1,0.26,0.74,0.092
2,0.32,0.68,0.357
3,0.33,0.67,0.198
4,0.30,0.70,0.086
5,0.29,0.71,0.086
6,0.30,0.70,0.198
7,0.33,0.67,0.088
8,0.26,0.74,0.194
9,0.25,0.75,0.085
10,0.28,0.72,0.115


Train Aggregated Result

In [16]:
apply(r$measures.train,2,mean)

iter      mmce       acc timetrain 
5.5000000 0.2845556 0.7154444 0.1499000

Test Aggregated Result

In [17]:
apply(r$measures.test,2,mean)

iter      mmce       acc timetrain 
   5.5000    0.2920    0.7080    0.1499

Run Time in seconds

In [18]:
r$runtime

[1] 1.698148

<h2>4. Prediction for new data</h2>

Read the data to predict

In [19]:
new.data = read.csv(file=sprintf('%s/%s', folder_name, file_name))
new.data = createDummyFeatures(obj = new.data, target = 'V21')

Search for the best model in crossvalidation and use it to score the incoming data

In [20]:
best.model = which.max(r$measures.test$acc)

In [21]:
pred = predict(r$models[[best.model]], newdata = new.data)

Prediction Result

In [22]:
pred

Prediction: 1000 observations
predict.type: response
threshold: 
time: 0.00
  truth response
1     1        2
2     2        1
3     1        2
4     1        1
5     2        1
6     1        1
... (1000 rows, 2 cols)


Cast result to data.frame to access the prediction

In [23]:
head(as.data.frame(pred))

truth,response
1,2
2,1
1,2
1,1
2,1
1,1
